In [95]:
import pandas as pd
import numpy as np
import pickle

In [96]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [97]:
PROFILE_DIR = "/mnt/storage/research/princeton/resource_estimation/gemm/3090/" 

In [98]:
with open(PROFILE_DIR + "profiling_df.pickle", "rb") as in_file:
    df = pickle.load(in_file)

In [99]:
with open(PROFILE_DIR + "error_files_list.pickle", "rb") as in_file:
    error_files = pickle.load(in_file)

In [100]:
df = df.loc[:,~df.columns.duplicated()].copy()
df = df.loc[:, df.columns.notna()]

In [101]:
### important columns for now
key_cols = ['M', 'K', 'N', 'Duration (nsecond)', 'Compute (SM) Throughput (%)', 'Memory Throughput (%)', 'Memory Throughput (byte/second)', 'DRAM Throughput (%)', 'Mem Busy (%)', 'Max Bandwidth (%)', 'Mem Pipes Busy (%)', 'Theoretical Occupancy (%)', 'Achieved Occupancy (%)', 
            'Elapsed Cycles (cycle)', 'SM Active Cycles (cycle)', 'SM: Pipe Tensor Cycles Active (%)',
            'Executed Ipc Active (inst/cycle)', 'Executed Ipc Elapsed (inst/cycle)', 'Issue Slots Busy (%)', 'Issued Ipc Active (inst/cycle)', 'SM Busy (%)', 'Tensor (All) (%)', 'ALU (%)', 'FMA (%)', 'FMA (FP16) (%)', 'FP64 (%)', 'FP64 (DMMA) (%)',  
            'L1/TEX Cache Throughput (%)', 'L2 Cache Throughput (%)', 'L1/TEX Hit Rate (%)', 'L2 Hit Rate (%)',
            'Active Warps Per Scheduler (warp)', 'Eligible Warps Per Scheduler (warp)', 'Theoretical Warps Per Scheduler (warp)', 'GPU Maximum Warps Per Scheduler (warp)', 'Active Warps Per Scheduler (warp)', 'Eligible Warps Per Scheduler (warp)', 'Warp Cycles Per Issued Instruction (cycle)', 'Warp Cycles Per Executed Instruction (cycle)',
            'Avg. Executed Instructions Per Scheduler (inst)', 'Executed Instructions (inst)', 'Avg. Issued Instructions Per Scheduler (inst)', 'Issued Instructions (inst)', 'Registers Per Thread (register/thread)', 'Shared Memory Per Block (byte/block)', 'Shared Memory Configuration Size (byte)', 'Driver Shared Memory Per Block (byte/block)', 'Dynamic Shared Memory Per Block (byte/block)', 'Static Shared Memory Per Block (byte/block)', 'Threads (thread)', 'Avg. Threads Executed (thread)', 'Avg. Predicated-On Threads Executed (thread)',
            'Theoretical Active Warps per SM (warp)', 'Achieved Active Warps Per SM (warp)',
            'L1 Wavefronts Shared Excessive (byte)', 'L2 Theoretical Sectors Global Excessive (byte)', 'Branch Instructions Ratio (%)',
            'SM Frequency (cycle/second)', 'DRAM Frequency (cycle/second)', 
            'Kernel Name', 'Block Size', 'Grid Size']

In [102]:
df_key = df[key_cols]

In [103]:
df_key["Duration (nsecond)"] = df_key["Duration (nsecond)"].apply(lambda x: x.replace(",", "")).astype(int)

/tmp/ipykernel_1667204/799244815.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_key["Duration (nsecond)"] = df_key["Duration (nsecond)"].apply(lambda x: x.replace(",", "")).astype(int)


In [104]:
max(df_key["Duration (nsecond)"])

605864256

In [105]:
df_key

,M,K,N,Duration (nsecond),Compute (SM) Throughput (%),Memory Throughput (%),Memory Throughput (byte/second),DRAM Throughput (%),Mem Busy (%),Max Bandwidth (%),Mem Pipes Busy (%),Theoretical Occupancy (%),Achieved Occupancy (%),Elapsed Cycles (cycle),SM Active Cycles (cycle),SM: Pipe Tensor Cycles Active (%),Executed Ipc Active (inst/cycle),Executed Ipc Elapsed (inst/cycle),Issue Slots Busy (%),Issued Ipc Active (inst/cycle),SM Busy (%),Tensor (All) (%),ALU (%),FMA (%),FMA (FP16) (%),FP64 (%),FP64 (DMMA) (%),L1/TEX Cache Throughput (%),L2 Cache Throughput (%),L1/TEX Hit Rate (%),L2 Hit Rate (%),Active Warps Per Scheduler (warp),Eligible Warps Per Scheduler (warp),Theoretical Warps Per Scheduler (warp),GPU Maximum Warps Per Scheduler (warp),Active Warps Per Scheduler (warp),Eligible Warps Per Scheduler (warp),Warp Cycles Per Issued Instruction (cycle),Warp Cycles Per Executed Instruction (cycle),Avg. Executed Instructions Per Scheduler (inst),Executed Instructions (inst),Avg. Issued Instructions Per Scheduler (inst),Issued Instructions (inst),Registers Per Thread (register/thread),Shared Memory Per Block (byte/block),Shared Memory Configuration Size (byte),Driver Shared Memory Per Block (byte/block),Dynamic Shared Memory Per Block (byte/block),Static Shared Memory Per Block (byte/block),Threads (thread),Avg. Threads Executed (thread),Avg. Predicated-On Threads Executed (thread),Theoretical Active Warps per SM (warp),Achieved Active Warps Per SM (warp),L1 Wavefronts Shared Excessive (byte),L2 Theoretical Sectors Global Excessive (byte),Branch Instructions Ratio (%),SM Frequency (cycle/second),DRAM Frequency (cycle/second),Kernel Name,Block Size,Grid Size
0,128,128,128,6528,0.79,3.30,"22,058,823,529.41",2.56,3.30,2.56,0.79,16.67,8.34,"8,597",308.68,0.79,0.81,0.03,20.64,0.83,20.64,17.32,16.18,2.62,0,0,0,32.36,3.30,0,72.94,0.99,0.21,2,12,0.99,0.21,4.80,4.89,62.49,"20,496",63.71,"20,896",100,"50,176","102,400","1,024","49,152",0,512,32,30,8,4.00,"1,664",0,0.01,"1,316,198,354.34","8,967,320,261.44",void cutlass::Kernel<cutlass_80_tensorop_s1688...,"(128, 1, 1)","(4, 1, 1)"
1,128,256,128,8416,1.03,3.88,"31,482,889,733.84",3.70,3.88,3.70,1.03,8.33,8.27,"10,932",435.04,1.03,0.84,0.03,21.27,0.85,22.96,22.96,18.57,1.65,0,0,0,45.93,3.88,0,59.06,1.07,0.21,1,12,1.07,0.21,5.16,5.24,91.12,"29,888",92.54,"30,352",102,"99,328","102,400","1,024","98,304",0,512,32,30,4,3.97,0,0,0.01,"1,298,156,572.51","8,861,850,443.60",void cutlass::Kernel<cutlass_80_tensorop_s1688...,"(128, 1, 1)","(4, 1, 1)"
2,128,512,128,12128,1.27,5.70,"48,221,635,883.91",5.70,4.98,5.70,1.21,8.33,7.99,"15,715",700.34,1.27,0.83,0.04,21.01,0.84,28.53,28.53,17.66,1.25,0,0,0,57.06,4.98,0,65.57,0.95,0.21,1,12,0.95,0.21,4.54,4.59,145.76,"47,808",147.17,"48,272",102,"99,328","102,400","1,024","98,304",0,512,32,30,4,3.84,0,0,0.01,"1,295,267,150.40","8,816,182,937.55",void cutlass::Kernel<cutlass_80_tensorop_s1688...,"(128, 1, 1)","(4, 1, 1)"
3,128,1024,128,19360,1.57,6.54,"56,310,743,801.65",6.54,5.77,6.54,1.36,8.33,8.08,"25,471","1,177.28",1.57,0.87,0.04,21.78,0.87,33.94,33.94,17.80,1.01,0,0,0,67.89,5.77,0,52.69,0.98,0.22,1,12,0.98,0.22,4.44,4.46,255.02,"83,648",256.44,"84,112",102,"99,328","102,400","1,024","98,304",0,512,32,30,4,3.88,0,0,0.01,"1,315,370,425.03","8,965,289,256.20",void cutlass::Kernel<cutlass_80_tensorop_s1688...,"(128, 1, 1)","(4, 1, 1)"
4,128,2048,128,32672,1.80,7.56,"67,200,783,545.54",7.56,6.47,7.56,1.48,8.33,8.32,"44,464","2,077.89",1.80,0.91,0.04,22.86,0.91,38.46,38.46,18.35,0.87,0,0,0,76.93,6.47,0,51.24,1.00,0.23,1,12,1.00,0.23,4.36,4.38,473.56,"155,328",474.98,"155,792",102,"99,328","102,400","1,024","98,304",0,512,32,30,4,3.99,0,0,0.01,"1,360,632,083.39","9,256,284,688.21",void cutlass::Kernel<cutlass_80_tensorop_s1688...,"(128, 1, 1)","(4, 1, 1)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.